In [1]:
import numpy
import pcbimport
from build123d import *
from jupyter_cadquery import open_viewer, set_defaults, show, get_defaults
set_defaults(theme="dark")
cv = open_viewer("CadQuery", anchor="right", glass=True)
set_defaults(reset_camera=False)

('invalid boolean data', [1251, 'locked', 'yes'])
('invalid boolean data', [3557, 'locked', 'yes'])
('invalid boolean data', [4796, 'locked', 'yes'])
('invalid boolean data', [12881, 'locked', 'yes'])
('invalid boolean data', [13975, 'locked', 'yes'])
('invalid boolean data', [14887, 'locked', 'yes'])
('invalid boolean data', [18197, 'locked', 'yes'])
('invalid boolean data', [25109, 'locked', 'yes'])
('invalid boolean data', [26875, 'locked', 'yes'])
('invalid boolean data', [30924, 'locked', 'yes'])
('invalid boolean data', [34412, 'locked', 'yes'])


Overwriting auto display for cadquery Workplane and Shape


In [203]:
AXIS_D = 2
TH = 1
H = 7


rodholder = Cylinder(AXIS_D/2+TH, H) - Pos(0,0, TH) * Cylinder(AXIS_D/2+0.2, H) \
+ Pos(5/2, (TH-0.2)/2+AXIS_D/2+0.2) * Box(5, TH-0.2, H) \
+ Pos(6/2 + 5-2, TH/2+AXIS_D/2+TH) * Box(6, TH, H)

rodholder.export_stl('/tmp/rodholder.stl')
rodholder

/tmp/ipykernel_1940918/3685180401.py:10: DeprecationWarning: Use the `export_stl` function instead
  rodholder.export_stl('/tmp/rodholder.stl')


In [204]:
# Main SURFACE
TA = (Align.CENTER, Align.CENTER, Align.MIN)
MAIN_SURFACE_Z = 1.5
MAIN_SURFACE_X = 99
MAIN_SURFACE_Y = 85

HOUSING_RADIUS = 10

# mounting studs for drive unit
STUD0 = [10.5 - 100/2, 40 - 85/2]
STUD1 = [84.5 - 100/2, 7.5 - 85/2]
STUD2 = [81.5 - 100/2, 79 - 85/2]

STUD_RUBBER_D = 7
STUD_H = 3
STUD_D = 3.5
STUD_OFFSET_Z = 0.6 # The stud starts 1mm above the main surface
STUD_DRILL_D = 1.5


OFFSET_DRIVE = Pos(2.5,0)

POS_DRIVE = OFFSET_DRIVE * Pos(47.5, 38) * Rot(180,0,90) # we need to rotate and shift the data from the drawing of the drive
DRIVE_OUTLINE = POS_DRIVE * extrude(make_face(import_svg('drawing_drive_mechanics_without_holes.svg')), 10, both=True)
STUDS = [OFFSET_DRIVE * Pos(p) for p in [STUD0, STUD1, STUD2]]

#+ Pos(MAIN_SURFACE_X/2-1/2-1, 0) * Box(1, MAIN_SURFACE_Y, MAIN_SURFACE_Z, align=TA) \
drive_base = fillet(Box(MAIN_SURFACE_X, MAIN_SURFACE_Y, MAIN_SURFACE_Z, align=TA).edges() | Axis.Z, HOUSING_RADIUS) \
- DRIVE_OUTLINE \
- Pos(0,0,STUD_OFFSET_Z) * STUDS * Cylinder((STUD_RUBBER_D+0.5)/2, 10, align=TA) \
+ Pos(0,0,STUD_OFFSET_Z) * STUDS * Cylinder(STUD_D/2, STUD_H, align=TA) \
- Pos(0,0,STUD_OFFSET_Z) * STUDS * Cylinder(STUD_DRILL_D/2, STUD_H, align=TA)

drive_base.export_stl('/tmp/drive_base.stl')
drive_base

/tmp/ipykernel_1940918/2470931162.py:34: DeprecationWarning: Use the `export_stl` function instead
  drive_base.export_stl('/tmp/drive_base.stl')


In [205]:
# housing shell
SHELL_Z = 10 # measured from Z=0
SHELL_TH = 1

# mount of main PCB
PCB_OFFSET_X = 1.5
PCB_OFFSET_Y = -6
STUDS_PCB_H = 3 #above main surface
PCB_TH = 0.8
SWITCH_TH = 1

PCB_POS = Pos(-MAIN_SURFACE_X/2+PCB_OFFSET_X, MAIN_SURFACE_Y/2+PCB_OFFSET_Y, MAIN_SURFACE_Z + STUDS_PCB_H)
shell = fillet(Box(MAIN_SURFACE_X, MAIN_SURFACE_Y, SHELL_Z, align=TA).edges() | Axis.Z, HOUSING_RADIUS)
shell = offset(shell, -SHELL_TH, shell.faces() | Axis.Z)

SWITCH_DRILL_D = 4.5
switch_drill = Rot(0,90,0) * Cylinder(SWITCH_DRILL_D/2, 10)
switch_drills = [PCB_POS * Pos(0,0,PCB_TH + SWITCH_TH/2) * Pos(p) for p in pcbimport.get_switches()] * switch_drill

usb_c_pos = pcbimport.get_component_location("USB1")
usb_c = PCB_POS * Pos(usb_c_pos[0], usb_c_pos[1], PCB_TH + 3/2) * Box(20, 9.5, 3.5)

microsd_pos = pcbimport.get_component_location("Card1")
microsd = PCB_POS * Pos(microsd_pos[0], microsd_pos[1], PCB_TH+2.5/2) * Box(30, 16, 3)

audio_pos = pcbimport.get_component_location("Audio1")
audio = PCB_POS * Pos(audio_pos[0], audio_pos[1], PCB_TH+5.6/2-0.2) * Rot(0,90,0) * Cylinder(5.6/2, 20)
shell = shell - switch_drills - usb_c - microsd - audio

shell

In [206]:
# mount of drive electronics
STUDS_DRIVE_POS = [[58-101/2+6, 82-85/2-1-2], [6-101/2+6, 14-85/2-2.5], [18-101/2+6, 63-85/2-2]]
STUDS_DRIVE_H = 3 #above main surface
STUDS_DRIVE_DRILL_D = 1.5

studs_drive = [Pos(p) for p in STUDS_DRIVE_POS] * (Cylinder(5/2, STUDS_DRIVE_H+MAIN_SURFACE_Z, align=TA) - Cylinder(STUDS_DRIVE_DRILL_D/2, STUDS_DRIVE_H+MAIN_SURFACE_Z, align=TA))


HINGE_X = 5
HINGE_Y = 5
HINGE_HOLDER_X = 5
HINGE_HOLDER_Y = 5
HINGE_SPHERE_D = 2.5
HINGE_HEIGHT = SHELL_Z - HINGE_SPHERE_D/2 - 1
HINGE0_X = -MAIN_SURFACE_X/2 + HOUSING_RADIUS + HINGE_HOLDER_X
HINGE1_X = MAIN_SURFACE_X/2 - HOUSING_RADIUS - HINGE_HOLDER_X - 12
hinge_opening = Box(HINGE_X+0.6, HINGE_Y+0.6, SHELL_Z, align=(Align.MIN, Align.MAX, Align.MIN))
hinge_holder = Box(HINGE_HOLDER_X, HINGE_HOLDER_Y, SHELL_Z, align=(Align.MAX, Align.MAX, Align.MIN)) + Pos(0, -HINGE_HOLDER_Y/2, HINGE_HEIGHT) * Sphere((HINGE_SPHERE_D)/2)

housing = drive_base + shell + studs_drive
#add hinges
housing = housing - Pos(HINGE0_X, MAIN_SURFACE_Y/2) * hinge_opening + Pos(HINGE0_X, MAIN_SURFACE_Y/2) * hinge_holder
housing = housing - Pos(HINGE1_X, MAIN_SURFACE_Y/2) * mirror(hinge_opening, Plane.YZ) + Pos(HINGE1_X, MAIN_SURFACE_Y/2) * mirror(hinge_holder, Plane.YZ)




STUDS_PCB_POS = pcbimport.get_mounting_holes()
STUDS_PCB_D = 3.5
STUDS_PCB_DRILL_D = 1.5
stud_pcb = Cylinder(STUDS_PCB_D/2, STUDS_PCB_H+MAIN_SURFACE_Z, align=TA) - Cylinder(STUDS_PCB_DRILL_D/2, STUDS_PCB_H+MAIN_SURFACE_Z, align=TA)
studs_pcb = [Pos(-MAIN_SURFACE_X/2+PCB_OFFSET_X, MAIN_SURFACE_Y/2+PCB_OFFSET_Y) * Pos(p) for p in STUDS_PCB_POS] * stud_pcb

#for mounting the back cap
STUDS_CAP_POS = [Pos(-MAIN_SURFACE_X/2+HOUSING_RADIUS-3, MAIN_SURFACE_Y/2-4, 0), Pos(MAIN_SURFACE_X/2-HOUSING_RADIUS+1, MAIN_SURFACE_Y/2-3, 0)]
studs_cap =  STUDS_CAP_POS * stud_pcb

housing += studs_pcb + studs_cap


#cutout for display
DISPLAY_Y = 32+0.5
DISPLAY_X = 9.3+0.5
DISPLAY_Z = 2
DISPLAY_AREA_Y = 23 # todo
DISPLAY_AREA_X = 5
DISPLAY_AREA_OFFSET_Y = 1
DISPLAY_AREA_OFFSET_X = 1.5
display_cutout = Pos(-MAIN_SURFACE_X/2+DISPLAY_Y/2+HOUSING_RADIUS, DISPLAY_Z+0.4, -SHELL_Z/2+DISPLAY_X/2+0.2) * (Plane((housing.faces() | Axis.Y)[0]) * (Box(DISPLAY_X, DISPLAY_Y, DISPLAY_Z, align=TA) + Pos(DISPLAY_X/2-DISPLAY_AREA_X/2-DISPLAY_AREA_OFFSET_X,DISPLAY_Y/2-DISPLAY_AREA_Y/2-DISPLAY_AREA_OFFSET_Y,0) * Box(DISPLAY_AREA_X, DISPLAY_AREA_Y, 50, align=TA)))

housing -= display_cutout



housing.export_stl('/tmp/housing.stl')
housing

/tmp/ipykernel_1940918/2917629096.py:55: DeprecationWarning: Use the `export_stl` function instead
  housing.export_stl('/tmp/housing.stl')


In [207]:
#lid
LID_INNER_H = 3.2 #previously, this was 4. check for collisions with disk!
LID_TH = 1

lid = fillet(Box(MAIN_SURFACE_X, MAIN_SURFACE_Y, LID_INNER_H+LID_TH, align=TA).edges() | Axis.Z, HOUSING_RADIUS)
lid = chamfer(lid.edges().group_by()[0], 3)

DISK_CENTER_X = -MAIN_SURFACE_X/2 + 41.8
DISK_CENTER_Y = -MAIN_SURFACE_Y/2 + 42.8
SPINDLE_D = 17

lid = lid + Pos(DISK_CENTER_X, DISK_CENTER_Y) * Cylinder(83/2, LID_INNER_H+LID_TH, align=TA)

lid = offset(lid, -LID_TH, lid.faces().group_by()[-1])

top_hinge_plane =  Plane.XY.offset(LID_INNER_H+LID_TH+SHELL_Z)

hinges_drawing =  top_hinge_plane * (Pos(HINGE0_X+0.2, MAIN_SURFACE_Y/2) * Rectangle(HINGE_X, HINGE_Y, align=(Align.MIN, Align.MAX)) + Pos(HINGE1_X-0.2, MAIN_SURFACE_Y/2) * Rectangle(HINGE_X, HINGE_Y, align=(Align.MAX, Align.MAX)))
hinges = extrude(hinges_drawing, until=Until.NEXT, target=lid, dir=(0,0,-1)) - Pos(0,MAIN_SURFACE_Y/2-HINGE_Y/2,HINGE_HEIGHT+LID_TH+LID_INNER_H) * Rot(0,90,0) * Cylinder((HINGE_SPHERE_D)/2, MAIN_SURFACE_X)

lid = lid + hinges - Pos(DISK_CENTER_X,DISK_CENTER_Y,0.5) * (Cylinder(82/2, 20, align=TA) + Pos(0,0,-0.3) * Cylinder(SPINDLE_D/2, 20, align=TA))


# Extension of hinge that presses onto the LID open switch
lid += Pos(HINGE0_X+0.2+HINGE_X/2, MAIN_SURFACE_Y/2-HINGE_Y, 3.5+LID_INNER_H) * Box(HINGE_X, 2.5, 7.5, align=TA)

#round off sides of the hinges
lid = fillet((lid.edges().group_by()[-1] | Axis.X).group_by(Axis.Y)[0], 3.8)
lid = fillet((lid.edges().group_by()[-1] | Axis.X).group_by(Axis.Y)[-1], 2)



lid.export_stl('/tmp/lid.stl')
lid

/tmp/ipykernel_1940918/1196160150.py:33: DeprecationWarning: Use the `export_stl` function instead
  lid.export_stl('/tmp/lid.stl')


In [274]:
#TODO

SWITCH_CAP_D = SWITCH_DRILL_D-0.6
SWITCH_CAP_H = 2.5
SWITCH_CAP_FLANGE_TH = 0.7
SWITCH_CAP_FLANGE_H = 0.5
SWITCH_INNER_DRILL_D = SWITCH_DRILL_D-2
SWITCH_INNER_DRILL_H = 0.4

coords_sorted = numpy.sort(numpy.array(pcbimport.get_switches())[:, 1])
coords_sorted -= coords_sorted[-1]
coords_sorted
switch_cap = Cylinder(SWITCH_CAP_D/2, SWITCH_CAP_H, align=TA) - Cylinder(SWITCH_INNER_DRILL_D/2, SWITCH_INNER_DRILL_H, align=TA)
bar = Pos(coords_sorted[0]/2, 0, 0) * Box(-coords_sorted[0], SWITCH_CAP_D, SWITCH_CAP_FLANGE_H, align=(Align.CENTER, Align.CENTER, Align.MAX))

switch_caps = bar + [Pos(p, 0, 0) for p in coords_sorted] * switch_cap - Pos(coords_sorted[0]/2, 0, SWITCH_CAP_FLANGE_H) * Box(-coords_sorted[0]+10, SWITCH_CAP_D - 2 * SWITCH_CAP_FLANGE_TH, SWITCH_CAP_FLANGE_H*2, align=(Align.CENTER, Align.CENTER, Align.MAX))

switch_caps = chamfer(switch_caps.edges().group_by()[-1], 0.4)
switch_caps.export_stl('/tmp/switch_caps.stl')
switch_caps

/tmp/ipykernel_873809/3780737958.py:19: DeprecationWarning: Use the `export_stl` function instead
  switch_caps.export_stl('/tmp/switch_caps.stl')


In [199]:
#bottom cap
CAP_Z = 2


shell_inner_edge = make_face(shell.edges().group_by()[0][8:])
shell_inner_edge = offset(shell_inner_edge, -0.3)

cap = extrude(shell_inner_edge, CAP_Z)

hinge_cutout0 = Pos(HINGE0_X, MAIN_SURFACE_Y/2) * hinge_opening + Pos(HINGE0_X, MAIN_SURFACE_Y/2) * hinge_holder
hinge_cutout0 = offset(hinge_cutout0, 0.5)                                                                         
hinge_cutout1 = Pos(HINGE1_X, MAIN_SURFACE_Y/2) * mirror(hinge_opening, Plane.YZ) + Pos(HINGE1_X, MAIN_SURFACE_Y/2) * mirror(hinge_holder, Plane.YZ)
hinge_cutout1 = offset(hinge_cutout1, 0.5)

AUDIO_X = 14.5+2
AUDIO_Y = 6.5+1
audio_pos = PCB_POS * Pos(MAIN_SURFACE_X-AUDIO_X/2, pcbimport.get_component_location("Audio1")[1])

BATTERY_X = 65 + 2
BATTERY_Y = 50 + 2

battery = Pos(-MAIN_SURFACE_X/2+SHELL_TH+1 + BATTERY_X/2,0,-10+CAP_Z-0.3) * Box(BATTERY_X, BATTERY_Y, 10, align=TA)

display = Pos(-MAIN_SURFACE_X/2+DISPLAY_Y/2+HOUSING_RADIUS,-MAIN_SURFACE_Y/2+SHELL_TH+1,0) * Box(DISPLAY_Y, 1, 10)

SCREW_HEAD_D = 3
STUDS_D = 4
STUDS_PCB_POS_SELECTED = [Pos(-MAIN_SURFACE_X/2+PCB_OFFSET_X, MAIN_SURFACE_Y/2+PCB_OFFSET_Y) * Pos(STUDS_PCB_POS[i]) for i in [6, 7]]

#we have two types of studs. One that joins directly with studs on the housing part and one that also screws through the PCB
#the latter needs to be shorter because the PCB needs to fit between housing studs and these cap studs
STUD_LONG_Z = SHELL_Z - STUDS_PCB_H-MAIN_SURFACE_Z - CAP_Z
STUD_SHORT_Z = STUD_LONG_Z - PCB_TH
studs_long = STUDS_CAP_POS * Cylinder(STUDS_D/2, STUD_LONG_Z, align=(Align.CENTER, Align.CENTER, Align.MAX))
studs_short = STUDS_PCB_POS_SELECTED * Cylinder(STUDS_D/2, STUD_SHORT_Z, align=(Align.CENTER, Align.CENTER, Align.MAX))
cb_long = STUDS_CAP_POS * CounterBoreHole(radius = 1.6/2, counter_bore_radius = SCREW_HEAD_D/2, counter_bore_depth=STUD_LONG_Z-1, depth=20)
cb_short = STUDS_PCB_POS_SELECTED * CounterBoreHole(radius = 1.6/2, counter_bore_radius = SCREW_HEAD_D/2, counter_bore_depth=STUD_SHORT_Z-1, depth=20)

USBC_POS = Pos(MAIN_SURFACE_X/2 - 10/2, MAIN_SURFACE_Y/2+PCB_OFFSET_Y+pcbimport.get_component_location("USB1")[1], -10+0.5)
usbc = USBC_POS * Box(10,10,10, align=TA)

RESET_POS = Pos(-MAIN_SURFACE_X/2+PCB_OFFSET_X, MAIN_SURFACE_Y/2+PCB_OFFSET_Y) * Pos(pcbimport.get_component_location("SW1"))
reset_hole = RESET_POS * Cylinder(3/2, 10)

cap = cap - hinge_cutout0 - hinge_cutout1 - audio_pos * Box(AUDIO_X, AUDIO_Y, 10) - battery - display + studs_long + studs_short - cb_long - cb_short - usbc - reset_hole

cap.export_stl('/tmp/cap.stl')
cap

/tmp/ipykernel_1940918/2128904772.py:47: DeprecationWarning: Use the `export_stl` function instead
  cap.export_stl('/tmp/cap.stl')


In [20]:
pcb = import_step('/tmp/minidisc.step')
pcb = Pos(-MAIN_SURFACE_X/2+PCB_OFFSET_X, MAIN_SURFACE_Y/2+PCB_OFFSET_Y, MAIN_SURFACE_Z+STUDS_PCB_H)*pcb
pcb.export_stl('/tmp/pcb.stl')
pcb

ValueError: STEP File /tmp/minidisc.step could not be loaded

In [130]:
assembly = Compound(label='assembly', children=[housing, Pos(0,0,-(LID_INNER_H+LID_TH)) * lid, Pos(0,0,SHELL_Z-CAP_Z) * cap])
assembly.color = Color('red')
assembly


100% ⋮————————————————————————————————————————————————————————————⋮ (3/3)  0.16s


In [15]:
pcbimport.get_component_location("USB1")

In [191]:
CounterBoreHole(radius = 1/2, counter_bore_radius = 2.5/2, counter_bore_depth=1, depth=20) + Cylinder(5, 20, align=TA)